# We start off by reading the data

In [ ]:
import shared

bl_dict = shared.bl_dict
bl_kurzel = shared.bl_kurzel

In [ ]:
from datetime import datetime
import pandas as pd
import plotly.graph_objects as go

dfs = {}
for kurzel in bl_kurzel:
    dfs[kurzel] = pd.read_pickle(f'data/df_vac_{kurzel}.pkl')

pd.options.plotting.backend = "plotly"


# colors
import plotly.express as px
from itertools import cycle

palette = shared.get_palette()


## Let's take a look

In [ ]:
dfs['DE'].tail(5)

In [ ]:
current_official_doses = dfs['DE'].tail(1)['dosen_kumulativ']
#current_official_doses.values[0]
current_official_doses

In [ ]:
to_plot = dfs['DE'].set_index('date')
fig = go.Figure()
def add_dosen_for_bl(kurzel):
    to_plot = dfs[kurzel].set_index('date')
    fig.add_trace(go.Scatter(x = to_plot.index,
                             y=to_plot['shots_sum'],
                             mode='lines',
                             name=f'{bl_dict[kurzel]}',
                             marker_color=next(palette),
                             line=dict(width=2.5)))

for kurzel in bl_kurzel:
    add_dosen_for_bl(kurzel)
palette = shared.get_palette()
fig.update_layout(
        width=1050,
        height=600,
        title='Absolute verteilte Impfdosen'
    )
shared.write_html(fig, 'shots-absolute')
fig.show()

### Average doses of the last days

In [ ]:
def avg_doses_of_last_x_days(last_days):
    data = []
    for i in range(1, last_days):
        data.append([i, int(dfs['DE'].tail(i)['shots_today'].sum() /i)])

    a = pd.DataFrame(columns=["Last x days", "Average vacs"], data=data)

    fig = go.Figure(data=[
        go.Bar(name='Three',x=a['Last x days'], y=a['Average vacs'])

    ])
    fig.update_layout(
        width=1000,
        height=350,
        title=f'Durchschnitt verteilte Impfdosen der letzten x Tage (Stand: {datetime.now().strftime("%Y-%m-%d")})'
    )
    shared.write_html(fig, f'avg-shots-last-{last_days}-days')
    fig.show()
    return a


In [ ]:
a = avg_doses_of_last_x_days(100)
a.head(7)

In [ ]:
a = avg_doses_of_last_x_days(365)



In [ ]:
def add_rolling(fig, df, days, kurzel):
    try:
        name = bl_dict[kurzel]
    except KeyError:
        name = kurzel
    fig.add_trace(go.Scatter(x = df.index,
                         y=df.shots_today.rolling(days).mean(),
                         mode='lines',
                         name=name,
                         marker_color=next(palette),
                         line=dict( width=4)))

def add_traces(fig, df):
    fig.add_trace(go.Scatter(x = df.index,
                             y=df.shots_today,
                             mode='markers',
                             name='Real',
                             marker=dict(
                                color='grey',
                                size=4,
                            ),
                        ))

palette = shared.get_palette()
fig7 = go.Figure()
for kurzel in bl_kurzel:
    to_plot = dfs[kurzel].set_index('date')    
    add_rolling(fig7, to_plot, 7, kurzel)


palette = shared.get_palette()
fig21 = go.Figure()
for kurzel in bl_kurzel:
    to_plot = dfs[kurzel].set_index('date')
    add_rolling(fig21, to_plot, 21, kurzel)

palette = shared.get_palette()
fig_BY = go.Figure()
to_plot = dfs['BY'].set_index('date')
add_traces(fig_BY, to_plot)
add_rolling(fig_BY, to_plot, 7, '7 Tage')
add_rolling(fig_BY, to_plot, 14, '14 Tage')
add_rolling(fig_BY, to_plot, 21, '21 Tage')

palette = shared.get_palette()
fig_DE = go.Figure()
to_plot = dfs['DE'].set_index('date')
add_traces(fig_DE, to_plot)
add_rolling(fig_DE, to_plot, 7, '7 Tage')
add_rolling(fig_DE, to_plot, 14, '14 Tage')
add_rolling(fig_DE, to_plot, 21, '21 Tage')


fig7.update_layout(
    width=1000,
    height=700,
    # yaxis_range=[0,400_000],
    title='Täglich verteilte Impfdosen mit rollendem 7 Tage Fenster'
)

fig21.update_layout(
    width=1000,
    height=700,
    # yaxis_range=[0,400_000],
    title='Täglich verteilte Impfdosen mit rollendem 21 Tage Fenster'
)

fig_BY.update_layout(
    width=1000,
    height=700,
    # yaxis_range=[0,400_000],
    title='Täglich verteilte Impfdosen mit verschiedenen rollenden Fenstern (Bayern)'
)

fig_DE.update_layout(
    width=1000,
    height=700,
    # yaxis_range=[0,400_000],
    title='Täglich verteilte Impfdosen mit verschiedenen rollenden Fenstern (Deutschland)'
)

shared.write_html(fig7, 'daily-shots-7-day-window')
shared.write_html(fig21, 'daily-shots-21-day-window')
shared.write_html(fig_BY, 'daily-shots-BY')
shared.write_html(fig_DE, 'daily-shots-DE')

fig7.show()
fig21.show()
fig_BY.show()
fig_DE.show()

## Vaccinations per weekday

In [ ]:

to_plot_all = dfs['DE'].groupby(["weekday_name"])['shots_today'].mean().sort_values()
to_plot_10 = dfs['DE'].tail(7*10).groupby(["weekday_name"])['shots_today'].mean().sort_values()
to_plot_5 = dfs['DE'].tail(7*5).groupby(["weekday_name"])['shots_today'].mean().sort_values()
to_plot_2 = dfs['DE'].tail(7*2).groupby(["weekday_name"])['shots_today'].mean().sort_values()
to_plot_1 = dfs['DE'].tail(7*1).groupby(["weekday_name"])['shots_today'].mean().sort_values()

palette = shared.get_palette()
fig = go.Figure(data=[
    go.Bar(name='complete timeline', y=to_plot_all, x=to_plot_all.index),
    go.Bar(name='10 weeks', y=to_plot_10, x=to_plot_10.index),
    go.Bar(name='5 weeks', y=to_plot_5, x=to_plot_5.index),
    go.Bar(name='2 weeks', y=to_plot_2, x=to_plot_2.index),
    go.Bar(name='last week', y=to_plot_1, x=to_plot_1.index),
])
fig.update_layout(
    width=1000,
    height=500,
    title='Verteilung der absoluten wöchentlichen Impfungen über die Wochentage',
)
fig.update_xaxes(categoryorder='array', categoryarray= ['Sat','Sun','Mon','Tue','Wed','Thu','Fri'])
shared.write_html(fig, "weekdays_total")
fig.show()

In [ ]:
def helper(number, sum):
    result = number / sum
    # print(f"Number is {number}, sum is {sum}, result is {result}")
    return result
    
def weekday_vac_proportion(df):
    df = df.groupby(["weekday_name"]).mean()
    df['shots_today']
    sum = df['shots_today'].sum()
    df['vac_proportion'] = df.apply(lambda x: helper(x['shots_today'], sum), axis=1)
    return df

In [ ]:
to_plot_all = weekday_vac_proportion(dfs['DE'])
to_plot_10 = weekday_vac_proportion(dfs['DE'].tail(7*10))
to_plot_5 = weekday_vac_proportion(dfs['DE'].tail(7*5))
to_plot_2 = weekday_vac_proportion(dfs['DE'].tail(7*2))
to_plot_1 = weekday_vac_proportion(dfs['DE'].tail(7*1))

palette = shared.get_palette()
fig = go.Figure(data=[
    go.Bar(name='complete timeline', y=to_plot_all['vac_proportion'], x=to_plot_all.index),
    go.Bar(name='10 weeks', y=to_plot_10['vac_proportion'], x=to_plot_10.index),
    go.Bar(name='5 weeks', y=to_plot_5['vac_proportion'], x=to_plot_5.index),
    go.Bar(name='2 weeks', y=to_plot_2['vac_proportion'], x=to_plot_2.index),
    go.Bar(name='1 weeks', y=to_plot_1['vac_proportion'], x=to_plot_1.index),

])
fig.update_layout(
    width=1000,
    height=500,
    title='Verteilung der wöchentlichen Impfungen über die Wochentage in Prozent',
)
fig.update_xaxes(categoryorder='array', categoryarray= ['Sat','Sun','Mon','Tue','Wed','Thu','Fri'])
shared.write_html(fig, "weekdays_prop")
fig.show()

## Vaccinations per calendar week

In [ ]:
df = dfs['DE']
to_plot_sum = df.groupby(['year_and_week']).sum()
to_plot_sum['year_and_week'] = to_plot_sum.index


In [ ]:
df = dfs['DE']
to_plot_sum = df.groupby(['year_and_week']).sum()
to_plot_sum.reindex()

In [ ]:
def vac_per_calendar_week(kurzel):
    df = dfs[kurzel]
    # tail(length - 5) removes 53rd calendar week
    to_plot_sum = df.groupby(['year_and_week']).sum()
    to_plot_mean = df.groupby(['year_and_week']).mean()
    to_plot_sum['year_and_week'] = to_plot_sum.index
    to_plot_mean['year_and_week'] = to_plot_mean.index
    to_plot_mean.year_and_week = to_plot_mean.year_and_week.apply(str)
    to_plot_sum.year_and_week = to_plot_sum.year_and_week.apply(str)


    fig = go.Figure(data=[
        go.Bar(name='Summe', y=to_plot_sum['shots_today'], x=to_plot_sum['year_and_week']),
        go.Bar(name='Durchschnitt', y=to_plot_mean['shots_today'], x=to_plot_mean['year_and_week']),
    ])
    fig.update_layout(
        width=1000,
        height=500,
        title=f'Impfungen pro Kalender Woche ({bl_dict[kurzel]})'
    )
    shared.write_html(fig, f'vac-per-calendar-week-{kurzel}')
    fig.show()

vac_per_calendar_week('DE')
vac_per_calendar_week('BY')
vac_per_calendar_week('HE')

In [ ]:
df = dfs['BY']
to_plot_sum = df.tail(len(df.index) - 9).groupby(["calendar_week"])['shots_today'].sum()
to_plot_sum.tail(20)
df.head(10)
df['date'].dt.year

df_filtered = df.loc[lambda x: df['date'].dt.year >= 2021]
df_filtered.head(10)


In [ ]:
def is_next_day_weekend(df):
    next_day = df.tail(1).date + pd.DateOffset(1)
    if next_day.dt.dayofweek.values[0] > 4:
        return True
    else:
        return False

DAYS_TO_LOOK_BACK = 2

import math

def guess_next_days_vacs(df, is_weekend):
    df_filtered = df[df['is_weekend']==is_weekend]
    mean = df_filtered.tail(DAYS_TO_LOOK_BACK)['shots_today'].values.mean()
    return math.ceil(mean)


speed_list = []

for kurzel in bl_kurzel:
    df = dfs[kurzel]
    weekdays = guess_next_days_vacs(df, False)
    weekends = guess_next_days_vacs(df, True)
    speed_list.append([bl_dict[kurzel], weekdays, weekends])
    #print(f"{kurzel} -> Last {DAYS_TO_LOOK_BACK} days mean for (Mon Tue Wed, Thu, Fri): {weekdays} and for (Sat, Sun): {weekends} ")

speed_df = pd.DataFrame(speed_list,columns = ['bundesland', 'speed_weekday','speed_weekend'])
speed_df = speed_df.sort_values('speed_weekday', ascending=False)

fig = px.bar(speed_df, x='bundesland', y='speed_weekday',labels={
                     "bundesland": "Bundesland",
                     "speed_weekday": "Tägliche Impfungen (Vermutung)",
        },)
fig.update_layout(
        width=700,
        height=400,
        title=f'Abschätzung täglicher Impfungen for Land und Bundesländer'
    )
fig.show()